## CD to curent directory

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [3]:
import sys
ROOT_PATH = '/content/drive/My Drive/university/projects/NLP/project/code/'
#ROOT_PATH = '/content/drive/My Drive/code/'
sys.path.append(ROOT_PATH)

%cd /content/drive/My Drive/university/projects/NLP/project/code/
#%cd /content/drive/My Drive/code/

/content/drive/My Drive/university/projects/NLP/project/code


## Imports

In [4]:
import torch
from torch import nn,optim
import torch.nn.functional as F
import os,random,csv
import numpy as np
!pip install empath
from utils.EarlyStopping import EarlyStopping
from pathlib import Path
from IPython.display import display
from torchtext import data
from torchtext.vocab import Vectors, GloVe
csv.field_size_limit(sys.maxsize)
from os import mkdir
from os.path import isdir
import pandas as pd
import itertools
from sklearn.metrics import precision_recall_fscore_support
import datetime

     |████████████████████████████████| 61kB 3.2MB/s 
  Created wheel for empath: filename=empath-0.89-cp36-none-any.whl size=57824 sha256=f61ebe2ac6b58d8bcfb4efb505b8f931080f02395fb07421393163ada2e193ef
  Stored in directory: /root/.cache/pip/wheels/84/ea/2f/2bc54d4f9985ce61753ebc5b00cb2df51d855589267c667308
Successfully built empath


## Models

### CNN

In [0]:
class CNNClassification(torch.nn.Module):
  """
  Implements a BiLSTM network with an embedding layer and
  single hidden layer.
  """
  def __init__(self, config, pretrained_embeddings, feat_names=[],
               in_channels=1,n_filters=128,filter_sizes=[3,4,5],stride=1,padding=0):
    super(CNNClassification, self).__init__()
    """
    Arguments
    ---------
    # batch_size : Size of the batch which is same as the batch_size of the data returned by the TorchText BucketIterator
    # hidden_size : Size of the hidden_state of the LSTM
    # vocab_size : Size of the vocabulary containing unique words
    # embed_size : Embeddding dimension of GloVe word embeddings
    # pretrained_embeddings : Pre-trained GloVe word_embeddings which 
    # in_channels : Number of input channels. Here it is 1 as the input data has dimension = (batch_size, num_seq, embedding_dim)
		# out_channels : Number of output channels after convolution operation performed on the input matrix
		# kernel_heights : A list consisting of 3 different kernel_heights. Convolution will be performed 3 times and finally results from each kernel_height will be concatenated.
    we will use to create our word_embedding look-up table 
    """
    # parameters
    self.n_classes = config.n_classes
    self.avg = config.avg
    self.embedding_dim = config.embedding_dim
    self.hidden_size = config.hidden_size
    self.nlayers = config.hidden_layer
    self.vocab_size, self.emb_dim = pretrained_embeddings.shape
    self.is_doc_embedding = config.is_doc_embedding
    self.embedding_type = config.embedding_type
    self.device = config.device
    self.in_channels = in_channels
    self.out_channels = n_filters
    self.kernel_heights = filter_sizes
    self.stride = stride
    self.padding = padding

    # layers:
    self._dropout = torch.nn.Dropout(config.dropout)
    self.word_embeddings = torch.nn.Embedding.from_pretrained(pretrained_embeddings,
                                                              freeze=True, 
                                                              padding_idx=pad_index)
    self.conv1 = nn.Conv2d(self.in_channels, self.out_channels, (self.kernel_heights[0], self.embedding_dim), self.stride, self.padding)
    self.conv2 = nn.Conv2d(self.in_channels, self.out_channels, (self.kernel_heights[1], self.embedding_dim), self.stride, self.padding)
    self.conv3 = nn.Conv2d(self.in_channels, self.out_channels, (self.kernel_heights[2], self.embedding_dim), self.stride, self.padding)
    self.hidden2tag = torch.nn.Linear(len(self.kernel_heights)*self.out_channels, self.n_classes) # Maps the output of the LSTM into tag space.
    self.meta_feat_names = []

  def conv_block(self, input_data, conv_layer):
    conv_out = conv_layer(input_data) # conv_out.size() = (batch_size, out_channels, dim, 1)
    activation = F.relu(conv_out.squeeze(3))# activation.size() = (batch_size, out_channels, dim1)
    max_out = F.max_pool1d(activation, activation.size()[2]).squeeze(2)# maxpool_out.size() = (batch_size, out_channels)
    return max_out
  
  #https://github.com/prakashpandey9/Text-Classification-Pytorch/blob/master/models/CNN.py

  def forward(self, text, text_lengths, **kwargs):
    """
    Parameters
		----------
		text: text of shape = (batch_size, num_sequences)
    		Returns
		-------
		Output of the linear layer containing logits for classes.
		logits.size() = (batch_size, output_size)
    """
  
    batch_size, seq_length = text.shape[0], text.shape[1] # dim: batch_size x seq_len
    #self.h, self.c_n = self.init_hidden(batch_size)


    # 1. embed the input - 
    embeds = self.word_embeddings(text.long()) # dim: (batch_size x seq_len x embedding_dim)
    #drop = self._dropout(embeds)
    embeds = embeds.unsqueeze(1) #dim: (batch_size x 1 x num_seq x embedding_dim)
    
    #2. convolution
    max_out1 = self.conv_block(embeds, self.conv1)
    max_out2 = self.conv_block(embeds, self.conv2)
    max_out3 = self.conv_block(embeds, self.conv3)
		
    #3. concatenate feature maps
    all_out = torch.cat((max_out1, max_out2, max_out3), 1) #dim: (batch_size, num_kernels*out_channels) = (batch_size, n_filters * len(filter_sizes))
    fc_in = self._dropout(all_out) #(batch_size, num_kernels*out_channels)
		
    # 4. Project to tag space
    tag_space = self.hidden2tag(fc_in)  # dim: (batch_size x num_tags)

    return tag_space

### CNN - MultiInputs

In [0]:
class CNN(torch.nn.Module):
  """
  Implements a BiLSTM network with an embedding layer and
  single hidden layer.
  """
  def __init__(self, config, pretrained_embeddings, in_channels=1,n_filters=128, 
               filter_sizes=[3,4,5],stride=1,padding=0):
    super(CNN, self).__init__()
    """
    Arguments
    ---------
    # batch_size : Size of the batch which is same as the batch_size of the data returned by the TorchText BucketIterator
    # hidden_size : Size of the hidden_state of the LSTM
    # vocab_size : Size of the vocabulary containing unique words
    # embed_size : Embeddding dimension of GloVe word embeddings
    # pretrained_embeddings : Pre-trained GloVe word_embeddings which 
    # in_channels : Number of input channels. Here it is 1 as the input data has dimension = (batch_size, num_seq, embedding_dim)
		# out_channels : Number of output channels after convolution operation performed on the input matrix
		# kernel_heights : A list consisting of 3 different kernel_heights. Convolution will be performed 3 times and finally results from each kernel_height will be concatenated.
    we will use to create our word_embedding look-up table 
    """
    # parameters
    self.n_classes = config.n_classes
    self.avg = config.avg
    self.embedding_dim = config.embedding_dim
    self.hidden_size = config.hidden_size
    self.nlayers = config.hidden_layer
    self.vocab_size, self.emb_dim = pretrained_embeddings.shape
    self.is_doc_embedding = config.is_doc_embedding
    self.embedding_type = config.embedding_type
    self.device = config.device
    self.in_channels = in_channels
    self.out_channels = n_filters
    self.kernel_heights = filter_sizes
    self.stride = stride
    self.padding = padding

    # layers:
    self._dropout = torch.nn.Dropout(config.dropout)
    self.word_embeddings = torch.nn.Embedding.from_pretrained(pretrained_embeddings.to(config.device),
                                                              freeze=True, 
                                                              padding_idx=pad_index)
    self.conv1 = nn.Conv2d(self.in_channels, self.out_channels, (self.kernel_heights[0], self.embedding_dim), self.stride, self.padding)
    self.conv2 = nn.Conv2d(self.in_channels, self.out_channels, (self.kernel_heights[1], self.embedding_dim), self.stride, self.padding)
    self.conv3 = nn.Conv2d(self.in_channels, self.out_channels, (self.kernel_heights[2], self.embedding_dim), self.stride, self.padding)
    self.hidden2tag = torch.nn.Linear(len(self.kernel_heights)*self.out_channels, self.n_classes) # Maps the output of the LSTM into tag space.
    self.meta_feat_names = []

  def conv_block(self, input_data, conv_layer):
    conv_out = conv_layer(input_data) # conv_out.size() = (batch_size, out_channels, dim, 1)
    activation = F.relu(conv_out.squeeze(3))# activation.size() = (batch_size, out_channels, dim1)
    max_out = F.max_pool1d(activation, activation.size()[2]).squeeze(2)# maxpool_out.size() = (batch_size, out_channels)
    return max_out

  def forward(self, text, text_lengths):
    """
    Parameters
		----------
		text: text of shape = (batch_size, num_sequences)
    		Returns
		-------
		Output of the linear layer containing logits for classes.
		logits.size() = (batch_size, output_size)
    """
  
    batch_size, seq_length = text.shape[0], text.shape[1] # dim: batch_size x seq_len
    #self.h, self.c_n = self.init_hidden(batch_size)


    # 1. embed the input - 
    embeds = self.word_embeddings(text.long()) # dim: (batch_size x seq_len x embedding_dim)
    #drop = self._dropout(embeds)
    embeds = embeds.unsqueeze(1) #dim: (batch_size x 1 x num_seq x embedding_dim)
    
    #2. convolution
    max_out1 = self.conv_block(embeds, self.conv1)
    max_out2 = self.conv_block(embeds, self.conv2)
    max_out3 = self.conv_block(embeds, self.conv3)
		
    #3. concatenate feature maps
    all_out = torch.cat((max_out1, max_out2, max_out3), 1) #dim: (batch_size, num_kernels*out_channels)
    fc_in = self._dropout(all_out) #(batch_size, num_kernels*out_channels)

    return fc_in

class CNNMultipleInputModel(torch.nn.Module):
  def __init__(self, config, pretrained_embeddings, feat_names, in_channels=1, n_filters=128, 
               filter_sizes=[3,4,5],stride=1,padding=0):
    super(CNNMultipleInputModel, self).__init__()
    self.n_classes = config.n_classes
    self.avg = config.avg
    self.NN_model = CNN(config, pretrained_embeddings, 
                        in_channels=1, n_filters=128, filter_sizes=[3,4,5],
                        stride=1, padding=0)
    self.NN_model.to(config.device)
    self.meta_feat_names = feat_names
    self.hidden_size = config.hidden_size
    self.concat_hidden = self.hidden_size+(n_filters*len(filter_sizes))
    self.input2hidden = torch.nn.Linear(len(self.meta_feat_names), self.hidden_size)
    self.hidden2tag = torch.nn.Linear(self.concat_hidden, self.n_classes)
    self._dropout = torch.nn.Dropout(config.dropout)
    

  def forward(self, text, text_lengths, meta_data_feat):
    #1. run CNN
    NN_output = self.NN_model(text, text_lengths)

    #2. convert to type float - (num_features X batch_size)
    NN_output = NN_output.type(torch.cuda.FloatTensor)
    meta_data_feat = meta_data_feat.type(torch.cuda.FloatTensor) 

    #3.add sigmoid function
    meta_data_feat = self.input2hidden(meta_data_feat.permute(1,0)) # (batch_size , hidden_size)
    meta_data_feat = torch.sigmoid(meta_data_feat)

    #4. concat text feaure with other features
    multi_input = torch.cat([NN_output, meta_data_feat], 1) # (batch_size, hidden_size*(n_filters*len(filter_sizes)))
    multi_input = self._dropout(multi_input)

    #5. hideen to tag
    tag_probs = self.hidden2tag(multi_input)
    return tag_probs
    

# Trainer

In [0]:
#%debug
class Trainer:
  def __init__(self, model, config):
    self.exp_num = config.exp_num
    self.device = config.device
    self.model = model
    self.model.to(self.device)
    self.model_name = config.model_name
    self.seed = config.seed
    self.lr = config.lr
    self.epochs = config.epochs
    self.save_model = config.save_model
    self.upload_model = config.upload_model
    self.model_weights_path = config.model_weights_path
    self.batch_size = config.batch_size
    self.momentum = config.momentum
    self.save_points = config.save_points
    self.best_loss = float("inf")
    self.best_acc = float("-inf")
    self.soft = torch.nn.Softmax(dim=1)
    self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr, weight_decay=config.weight_decay)
    self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=self.optimizer,
                                                  mode='min', factor=0.1,
                                                  patience=10, verbose=1, 
                                                  eps=1e-4)
    self.lambda_regu = config.lambda_regu
    torch.manual_seed(self.seed)
    if torch.cuda.is_available():
      torch.cuda.manual_seed_all(self.seed)
    else: 
      torch.cuda.manual_seed(self.seed)
    torch.cuda.seed_all()
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    self.weight = torch.tensor(config.class_weight).to(self.device)
    self.criterion = torch.nn.CrossEntropyLoss(weight=self.weight,reduction='sum')
    # initialize the early_stopping object
    self.early_stopping = EarlyStopping(patience=config.patience, verbose=True)
    self.results = {}
    self.sgd_spv_matrix = {}
    self.eps = config.eps
    self.avg = config.avg

  def calc_accuracy(self, log_ps, labels):
    self.model.eval()
    ps = torch.exp(log_ps)
    top_p, top_class = ps.topk(1, dim=1)
    equals = top_class == labels.view(*top_class.shape)
    acc = torch.mean(equals.type(torch.FloatTensor))
    self.model.train()
    return acc

  def save_best_model(self, loss_val, acc_val, p, r, f, epoch, measure):
    weights_path_model = Path(f"{self.model_weights_path}/best_{measure}_{self.model_name}_{self.exp_num}_seed_{self.seed}.pth")
    print('saving_model best model: ')
    torch.save({'model_state_dict': self.model.state_dict(),
                'optimizer_state_dict': self.optimizer.state_dict(),
                'loss': loss_val,
                'accuracy': acc_val,
                'p':p,
                'r':r,
                'f':f}, weights_path_model)


  def save_checkpoint(self, loss_val, acc_val, p, r, f, epoch):
    weights_path_model = Path(f"{self.model_weights_path}/{self.model_name}_{epoch}_seed_{self.seed}.pth")
    if (epoch in self.save_points) and (not weights_path_model.exists()) and self.save_model:
        print('saving_model: ')
        torch.save({'model_state_dict': self.model.state_dict(),
                    'optimizer_state_dict': self.optimizer.state_dict(),
                    'loss': loss_val,
                    'accuracy': acc_val,
                    'p':p,
                    'r':r,
                    'f':f}, weights_path_model)

  def load_checkpoint(self, weights_path, epoch):
    checkpoint = torch.load(weights_path)
    self.model.load_state_dict(checkpoint['model_state_dict'])
    self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    self.model.to(self.device)
    loss = checkpoint['loss']
    accuracy = checkpoint['accuracy']
    # p = checkpoint['p']
    # r = checkpoint['r']
    # f = checkpoint['f']
    print(f"Uploaded weights succesfuly at epoch number {epoch}")
    return loss, accuracy

  def record(self, epoch, **kwargs):
    epoch = "{:02d}".format(epoch)
    temp = f"| epoch   # {epoch} :"
    for key, value in kwargs.items():
      key = f"{self.model_name}_{key}"
      if not self.results.get(key):
        self.results[key] = []
      self.results[key].append(value)
      val = '{:.4f}'.format(np.round(value, 4))
      temp += f"{key} : {val}      |       "
    print(temp)

  def clip_gradient(self, clip_value):
    params = list(filter(lambda p: p.grad is not None, self.model.parameters()))
    for p in params:
        p.grad.data.clamp_(-clip_value, clip_value)

  def fit(self, trainloader, validloader, config):
    is_loaded = False
    epoch = 1
    while epoch<self.epochs+1:
      retulst = []
      l_epoch = -1 if epoch==1 else epoch-1
      for i in self.save_points[::-1]:
        weights_path = Path(f"{self.model_weights_path}/{config.model_name}_{i}_seed_{self.seed}.pth")
        if (weights_path.exists()) and (self.upload_model) and (epoch<i) and (not is_loaded):
          epoch_train_loss, epoch_train_acc = self.load_checkpoint(weights_path, i)
          epoch = i+1
          l_epoch = i
          is_loaded = True
          break
      epoch_train_loss, epoch_train_acc, (p_train, r_train, f_train) = self.train_model(trainloader, epoch)
      epoch_valid_loss, epoch_valid_acc, (p_valid, r_valid, f_valid) = self.eval_model(validloader, epoch)
      if epoch_valid_loss<self.best_loss:
        self.best_loss = epoch_valid_loss
        self.save_best_model(epoch_valid_loss, epoch_valid_acc, p_valid, r_valid, 
                      f_valid, epoch, 'val_loss')
      if epoch_valid_acc>self.best_acc:
        self.best_acc = epoch_valid_acc
        self.save_best_model(epoch_valid_loss, epoch_valid_acc, p_valid, r_valid, 
                      f_valid, epoch, 'val_acc')
      retulst.append([self.exp_num, self.model_name, datetime.datetime.now(), config.exp, epoch, epoch_train_loss, 
                     epoch_valid_loss,epoch_train_acc, 
                     epoch_valid_acc,p_train, p_valid, r_train, r_valid, f_train, f_valid])
      df = pd.DataFrame(retulst)
      df.to_csv(f'results/multi_input_{config.dataset}_results.csv', mode='a', header=False, index=False)
      self.scheduler.step(epoch_valid_loss)
      self.record(epoch, train_loss=epoch_train_loss,
                  validation_loss=epoch_valid_loss,
                  train_accuracy=epoch_train_acc,
                  validation_accuracy=epoch_valid_acc,
                  train_p=p_train,
                  validation_p=p_valid,
                  train_r=r_train,
                  validation_r=r_valid,
                  train_f=f_train,
                  validation_f=f_valid)
      self.save_checkpoint(epoch_train_loss, epoch_train_acc, p_train, r_train, 
                           f_train, epoch)
      self.early_stopping(epoch_valid_loss)
      if self.early_stopping.early_stop:
        print("Early stopping")
        break
      epoch += 1

  def caclculate_p_r_f(self, prediction):
    self.model.eval()
    predic_prob = self.soft(prediction)
    predicted_label = torch.argmax(predic_prob,axis=1)
    return predicted_label.cpu().detach().numpy()

  def train_model(self, train_iter, epoch):
    total_epoch_loss, total_epoch_acc, steps = 0, 0, 0
    self.model.train()
    train_loss, counter = 0, 0
    for idx, batch in enumerate(train_iter):
      features_data = []
      text, text_lengths = batch.text
      target = batch.label

      # add meta features
      for i, feat in enumerate(self.model.meta_feat_names):
        features_data.append(getattr(batch, feat))
      if len(features_data)!=0:
        features_data = torch.stack(features_data)
        features_data = features_data.to(self.device)

      target = torch.autograd.Variable(target).long()
      text = text.to(self.device)
      target = target.to(self.device)
      self.optimizer.zero_grad()
      prediction = self.model(text=text, text_lengths=text_lengths, meta_data_feat=features_data)
      loss = self.criterion(prediction, target)

      # calculate accuracy
      num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).float().sum()
      acc = 100.0 * num_corrects / len(batch)

      predicted_label = self.caclculate_p_r_f(prediction)
      (p,r,f,_) = precision_recall_fscore_support(target.cpu().detach().numpy() , predicted_label , average=self.avg)

      self.model.train()
      loss.backward()
      self.clip_gradient(1)
      self.optimizer.step()
      total_epoch_loss += loss.item() / len(batch)
      total_epoch_acc += acc.item()
      
    return total_epoch_loss / len(train_iter), total_epoch_acc / len(train_iter), (p,r,f)

  def eval_model(self, val_iter, epoch):
    total_epoch_loss, total_epoch_acc = 0, 0
    self.model.eval()
    with torch.no_grad():
      for idx, batch in enumerate(val_iter):
        features_data = []
        text, text_lengths = batch.text
        target = batch.label

        # add meta features
        for i, feat in enumerate(self.model.meta_feat_names):
          features_data.append(getattr(batch, feat))
        if len(features_data)!=0:
          features_data = torch.stack(features_data)
          features_data = features_data.to(self.device)

        target = torch.autograd.Variable(target).long()
        text = text.to(self.device)
        target = target.to(self.device)
        prediction = self.model(text=text, text_lengths=text_lengths, meta_data_feat=features_data)
        loss = self.criterion(prediction, target)

        # calculate accuracy
        num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).sum()
        acc = 100.0 * num_corrects / len(batch)
        total_epoch_loss += loss.item()  / len(batch)
        total_epoch_acc += acc.item()

        predicted_label = self.caclculate_p_r_f(prediction)
        (p,r,f,_) = precision_recall_fscore_support(target.cpu().detach().numpy() , predicted_label , average=self.avg)


    return total_epoch_loss / len(val_iter), total_epoch_acc / len(val_iter), (p,r,f)

# Run

## Configuration

In [0]:
class Config:
  """Holds model hyperparams and data information.
  The config class is used to store various hyperparameters and dataset
  information parameters. Model objects are passed a Config() object at
  instantiation.
  """
  def __init__(self, **kwargs):
    for key, value in kwargs.items():
      setattr(self, key, value)

  def add_attributes(self, **kwargs):
    for key, value in kwargs.items():
      setattr(self, key, value)

def create_directories(l):
  for directory_path in l:
    if not (isdir(directory_path)):
      mkdir(directory_path)


def get_base_config():
  ROOT_PATH = 'models/multi_input/'
  model_weights_dir = f"{ROOT_PATH}model_weights"
  graphs_dir = f"{ROOT_PATH}graphs"
  SAVE_FIGS = True
  ####################################################################
  BATCH_SIZE = 128 
  embedding_length = 100
  epochs = 50
  ####################################################################
  # model consistency options
  SAVE_TO_CHECKPOINTS = True # if true, saves model.name_epcoch file into the weights folder
  LOAD_CHECKPOINTS = False # # if true, every epoch tries to load pretrained weights
  ####################################################################
  # if needed, can be modified to upload the 'best model'
  return Config(seed=2,
                # learning
                batch_size=BATCH_SIZE,
                eps=0.00001, patience=5, weight_decay=5e-4,momentum=0.9,lr=0.001,
                #model
                hidden_layer=1,
                seq_max_len=500, embedding_dim=embedding_length,
                is_doc_embedding = False, embedding_type ='avg',
                #save
                save_points=np.arange(1,(epochs/10)+1)*10,
                save_model=SAVE_TO_CHECKPOINTS,
                upload_model=LOAD_CHECKPOINTS,
                #path
                model_weights_path=model_weights_dir,
                graphs_path = graphs_dir,
                device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))

config = get_base_config()

### configure dataset

In [0]:
# define configuration for the different datasets
datasets = {"combined":{"root_path_data": r"data/preprocessed/ny_guardian_kaggle/combined_data_tokenized_shuffled.csv",
                      "origin_cols": ['label', 'date', 'source', 'text_original', 'title', "text"],
                      "n_classes":2, "has_valid":False, "class_weight":[0.3, 0.7], "avg":"binary","dropout":0.1},
            "liar":{"root_path_data": r"data/preprocessed/liar_dataset/",
                      "origin_cols": ['ID', 'label', 'text_original', 'class', 'text'],
                      "n_classes":6, "has_valid":True, "class_weight":[2/15, 2/15, 2/15, 2/15, 2/6, 2/15],
                      "avg":"micro","dropout":0.5},
            "liar_binary":{"root_path_data": r"data/preprocessed/liar_dataset/",
                      "origin_cols": ['ID', 'label_all', 'text_original', 'label', 'text'],
                      "n_classes":2, "has_valid":True, "class_weight":[0.3, 0.7],
                      "avg": "binary","dropout":0.3},
            "Buzz+PoliticalFact":{"root_path_data": r"data/preprocessed/Buzz+PoliticalFact/combined_data_tokenized.csv",
                      "origin_cols": ['text_original', 'title', 'url', 'publish_date', 'source', 'label','text'],
                      "n_classes":2, "has_valid":False, "class_weight":[0.5, 0.5], "avg":"binary","dropout":0.5}}          

### configure Features

In [0]:
# define the columns names for each type of features
lexical = ["article_length", "total_sentences", 
"avg_sent_length", "word_count", "number_count",
"avg_word_length", "count_pos_adj", "count_pos_sl", 
"count_exclamation", "count_date", "allCapsCount"]

category = ["catLaw","catCommunication", "catCrime","catPayment",
                       "catPhone", "catBanking", "catWar", "catEconomics",
                       "catPolitics", "catLeader", "catSocialMedia", "catSchool", 
                       "catGovernment","catMoney","catWork", "catSpeaking",
                       "catInternet", "catBusiness"]

sentiment = ["sentNeutral","sentPos","sentCompound"]

### configure features model

In [0]:
def get_model_features(model_name="CNN"):
  if model_name=='CNN':
    return {"":[]}
  else: 
    return {"lexical":lexical, "sentiment":sentiment, "category":category}

## Load data and Embedding

In [0]:
def load_dataset(path, glove_dim, doc_length, SEED, device, origin_cols, has_valid=False,debug=False):
    """
    tokenizer : Breaks sentences into a list of words. If sequential=False, no tokenization is applied
    Field : A class that stores information about the way of preprocessing
    fix_length : An important property of TorchText is that we can let the input to be variable length, and TorchText will
                 dynamically pad each sequence to the longest sequence in that "batch". But here we are using fix_length which
                 will pad each sequence to have a fix length of doc_length.

    build_vocab : It will first make a vocabulary or dictionary mapping all the unique words present in the train_data to an
                  idx and then after it will use GloVe word embedding to map the index to the corresponding word embedding.

    vocab.vectors : This returns a torch tensor of shape (vocab_size x embedding_dim) containing the pre-trained word embeddings.
    BucketIterator : Defines an iterator that batches examples of similar lengths together to minimize the amount of padding needed.

    """

    # loading custom dataset
    # initialize glove embeddings
    
    TEXT = data.Field(sequential=True, tokenize=lambda x: x.split(" "),
                      lower=False, include_lengths=True,
                      batch_first=True, fix_length=None, init_token="<SOA>", eos_token="<EOA>")
    NUMERICAL_FLOAT = data.Field(use_vocab=False,sequential=False, dtype=torch.float64)
    NUMERICAL_INT = data.Field(use_vocab=False,sequential=False, dtype=torch.int64)
    LABEL = data.LabelField(use_vocab=False,sequential=False)

    # label, date, source, text, title                    
    preprocessed_fields = [("article_length", NUMERICAL_FLOAT),
              ("total_sentences", NUMERICAL_FLOAT),("avg_sent_length", NUMERICAL_FLOAT),
              ("word_count", NUMERICAL_FLOAT), ("number_count", NUMERICAL_FLOAT),
              ("avg_word_length", NUMERICAL_FLOAT), ("count_pos_adj", NUMERICAL_FLOAT), 
              ("count_pos_sl", NUMERICAL_FLOAT), ("count_exclamation", NUMERICAL_FLOAT), 
              ("count_date", NUMERICAL_FLOAT), ("allCapsCount", NUMERICAL_FLOAT),
              ("catLaw", NUMERICAL_FLOAT),("catCommunication", NUMERICAL_FLOAT),
              ("catCrime", NUMERICAL_FLOAT),("catPayment", NUMERICAL_FLOAT),
              ("catPhone", NUMERICAL_FLOAT), ("catBanking", NUMERICAL_FLOAT),
              ("catWar", NUMERICAL_FLOAT),("catEconomics", NUMERICAL_FLOAT),
              ("catPolitics", NUMERICAL_FLOAT), ("catLeader", NUMERICAL_FLOAT), 
              ("catSocialMedia", NUMERICAL_FLOAT),("catSchool", NUMERICAL_FLOAT),
              ("catGovernment", NUMERICAL_FLOAT),("catMoney", NUMERICAL_FLOAT), 
              ("catWork", NUMERICAL_FLOAT),("catSpeaking", NUMERICAL_FLOAT),
              ("catInternet", NUMERICAL_FLOAT),("catBusiness", NUMERICAL_FLOAT),
              ("sentNeg", NUMERICAL_FLOAT),("sentNeutral", NUMERICAL_FLOAT),
              ("sentPos", NUMERICAL_FLOAT), ("sentCompound", NUMERICAL_FLOAT)]

    dict_fields = {'label':LABEL,
                   'text':TEXT}
    original_fields = [(col, dict_fields.get(col)) for col in origin_cols]
    fields = original_fields + preprocessed_fields
    
    if not has_valid:
      train_data = data.TabularDataset(path=path, format='csv', fields=fields, skip_header=True)
      train_data, valid_data = train_data.split(split_ratio=0.8, random_state=random.seed(SEED))
    else:
      train_data, valid_data = data.TabularDataset.splits(path=path, train="train_tokenized.csv", 
                                                          validation="valid_tokenized.csv", format="csv",
                                                          skip_header=True, fields=fields)

    # Further splitting of training_data to create new training_data & validation_data
    train_iter, valid_iter = data.BucketIterator.splits((train_data, valid_data), batch_size=128,
                                                        sort_key=lambda x: len(x.text), repeat=False,
                                                        shuffle=True)
    # Initialized embedding
    if not os.path.exists(".vector_cache"):
        os.mkdir(".vector_cache")

    TEXT.build_vocab(train_data, min_freq=3,
                     vectors=GloVe(name='6B',
                                   dim=glove_dim))
    LABEL.build_vocab(train_data)

    word_embeddings = TEXT.vocab.vectors  # pretrained_embedding
    vocab_size = len(TEXT.vocab)

    if debug:
      # printing
      print("Length of Text Vocabulary: " + str(len(TEXT.vocab)))  # No. of unique tokens in text
      print("Vector size of Text Vocabulary: ", TEXT.vocab.vectors.size())
      print("Label Length: " + str(len(LABEL.vocab)))  # No. of unique tokens in label
      print(TEXT.vocab.freqs.most_common(10))  # Commonly used words
      print(TEXT.vocab.stoi)  # Word dictionary

    return vocab_size, word_embeddings, train_iter, valid_iter, TEXT.vocab.stoi['<pad>']

## Experiments

In [0]:
create_directories([config.graphs_path,config.model_weights_path,'results'])

In [0]:
# run model
model_name_list = ["CNN",
                  "MultiCNN"]
# define model
models = {"CNN": CNNClassification,
          "MultiCNN": CNNMultipleInputModel}
lr = [0.001]
hidden_size = [100]
hp_comb = list(itertools.product(lr, hidden_size))
dataset_list = ['combined']
for model_name in model_name_list:
  features = get_model_features(model_name=model_name)
  for selected_dataset in dataset_list:
    config.add_attributes(**datasets[selected_dataset], dataset=selected_dataset)
    #load dataset and embedding
    vocab_size, word_embeddings, train_iter, valid_iter, pad_index =\
        load_dataset(config.root_path_data, config.embedding_dim, 
                    config.seq_max_len, config.seed, config.device,
                    config.origin_cols, config.has_valid)
    for exp_num in np.arange(3):
      for l, h in hp_comb:
        for feat_name, feat in features.items():
          print('lr', l, 'hidden', h, 'drop_ratio', config.dropout)
          exp_name = f'{model_name}_{feat_name}_{selected_dataset}'
          config.add_attributes(model_name=exp_name, 
                                lr = l,
                                seed = 2,
                                epochs=50,
                                weight_decay = 0.00005,
                                hidden_size=h,
                                lambda_regu=0.0,
                                exp=f'{l},{h},{config.dropout}',
                                exp_num=exp_num)
          model = models[model_name](config=config, pretrained_embeddings=word_embeddings, feat_names=feat)
          print(f"{'#'*50} \n running {config.model_name} experiment\n{'#'*50} \n") 
          trainer = Trainer(model, config)
          trainer.fit(train_iter, valid_iter, config)

lr 0.001 hidden 100 drop_ratio 0.1
################################################## 
 running CNN__combined experiment
################################################## 



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


saving_model best model: 
saving_model best model: 
| epoch   # 01 :CNN__combined_train_loss : 0.1299      |       CNN__combined_validation_loss : 0.0765      |       CNN__combined_train_accuracy : 87.1642      |       CNN__combined_validation_accuracy : 94.0996      |       CNN__combined_train_p : 0.8261      |       CNN__combined_validation_p : 0.8571      |       CNN__combined_train_r : 0.8837      |       CNN__combined_validation_r : 0.7500      |       CNN__combined_train_f : 0.8539      |       CNN__combined_validation_f : 0.8000      |       
saving_model best model: 
saving_model best model: 
| epoch   # 02 :CNN__combined_train_loss : 0.0534      |       CNN__combined_validation_loss : 0.0491      |       CNN__combined_train_accuracy : 95.2980      |       CNN__combined_validation_accuracy : 95.1833      |       CNN__combined_train_p : 0.8913      |       CNN__combined_validation_p : 0.8750      |       CNN__combined_train_r : 1.0000      |       CNN__combined_validation_r : 0.

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


saving_model best model: 
saving_model best model: 
| epoch   # 01 :CNN__combined_train_loss : 0.1227      |       CNN__combined_validation_loss : 0.0670      |       CNN__combined_train_accuracy : 87.6502      |       CNN__combined_validation_accuracy : 93.8312      |       CNN__combined_train_p : 0.7451      |       CNN__combined_validation_p : 0.7000      |       CNN__combined_train_r : 0.9744      |       CNN__combined_validation_r : 0.8750      |       CNN__combined_train_f : 0.8444      |       CNN__combined_validation_f : 0.7778      |       
saving_model best model: 
saving_model best model: 
| epoch   # 02 :CNN__combined_train_loss : 0.0492      |       CNN__combined_validation_loss : 0.0462      |       CNN__combined_train_accuracy : 95.8205      |       CNN__combined_validation_accuracy : 96.4854      |       CNN__combined_train_p : 0.8788      |       CNN__combined_validation_p : 0.8750      |       CNN__combined_train_r : 0.9355      |       CNN__combined_validation_r : 0.